In [8]:
import pickle
import numpy as np
import cv2
import sys
import os
import glob
from sklearn import svm
from scipy.stats import mode
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm

In [9]:
featuresLimit = 15000

In [10]:

def featureExtraction( videoPath, actionName, type):



    # Set frame path, if jpeg directory doesn't exist , take images from video dir
    framePath = videoPath
    if os.path.exists( framePath + "/jpeg") :
        framePath += "/jpeg/"
    # Extract feature
    pickle_name='video_summary.pkl'
    file_path=os.path.join(videoPath,pickle_name)
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
        imageFrames=data['images_files']
    #print "DEBUG: Image Frames - ", imageFrames
    frameCount = 0
    frameIndex = 0

    # Feature List for a video
    videoFeatures  = []
    print(videoPath)
    for iFrame in imageFrames:
        
        frameIndex += 1
        # Only take alternate frames
        if  (frameIndex % 2)  == 0 :
            continue

        # Read Frame
        frame = cv2.imread(iFrame)
        # Create SIFT object
        sift = cv2.xfeatures2d.SIFT_create()
        gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        kp, des = sift.detectAndCompute(gray, None)

        fIndex = 0 # Feature Index
        if(des is None):
            continue
        for d in des:
            # Insert Label Only for Training
            if type == "Trng":
                d = np.insert(d, 0, sportsActionTag[actionName])

            videoFeatures.append(d)
            if fIndex >= featuresLimit:
                break
            fIndex += 1


        if frameCount >= 23:
            break
        frameCount += 1



    print(" \t\tFrame Count:{0}".format(frameCount))
    #print "Video Features: ", videoFeatures

    return videoFeatures

In [11]:
sportsActionTag = {
    'no_fight': 0,
    'fight':1
}

In [12]:
videoPath='./data/raw_frames/violentflow/audience_violence__Riot_police_attack_peaceful_protesters_at_G20_Climate_Camp__bristlekrs__t244-zEENSs'
sportsActionName='fight'

In [13]:
videoFeatures = featureExtraction(videoPath , sportsActionName, 'Trng')

./data/raw_frames/violentflow/audience_violence__Riot_police_attack_peaceful_protesters_at_G20_Climate_Camp__bristlekrs__t244-zEENSs
 		Frame Count:11


In [14]:
def getListOfDir(path):
    # Read each sport action directory
    dirs  = os.listdir(path)

    sportsActionsCount = 0
    filtered_dir  = []
    # Remove . .. and hidden directory
    for dir in dirs:
        if not dir.startswith("."):
            filtered_dir.append(dir)

    return filtered_dir

In [15]:
sportsActionPath='./data/raw_frames/violentflow'
sportsActionList = getListOfDir( sportsActionPath )

In [16]:
firstActionFlag = 0
videoFeatures = []
videoCount=1
for sportsActionName in tqdm(sportsActionList):
#     if(videoCount==10):
#         break
    sportsActionDir = sportsActionPath + "/" + sportsActionName
    if('violence' in sportsActionName):
        sportsActionName='fight'
    else:
        sportsActionName='no_fight'
        
    videoFeatures = featureExtraction(sportsActionDir , sportsActionName, 'Trng')
#     print(len(videoFeatures))
    # Put together all the videos
    if firstActionFlag == 0:
        sportsActionFeatures = videoFeatures
        firstActionFlag = 1
    else:
        sportsActionFeatures = np.concatenate( (sportsActionFeatures, videoFeatures), axis=0)

    videoCount += 1
    

  0%|          | 0/246 [00:00<?, ?it/s]

./data/raw_frames/violentflow/audience_violence__Riot_police_attack_peaceful_protesters_at_G20_Climate_Camp__bristlekrs__t244-zEENSs


  0%|          | 1/246 [00:00<01:57,  2.08it/s]

 		Frame Count:11
./data/raw_frames/violentflow/balcony_football__birmingham_promoted_to_premier_from_balcony_town_hall__clippo67_


  1%|          | 2/246 [00:01<02:10,  1.87it/s]

...

 99%|█████████▉| 244/246 [02:37<00:01,  1.14it/s]

./data/raw_frames/violentflow/stadium_violence__soccer_riot_compilation__qqcolaman__URwiZLZipeg
 		Frame Count:15


100%|█████████▉| 245/246 [02:38<00:00,  1.22it/s]

./data/raw_frames/violentflow/stadium_violence__stadium_violence__Barras_Argentinas__alebb77__en5llVbjq20
 		Frame Count:10


100%|██████████| 246/246 [02:39<00:00,  1.31it/s]


In [17]:
sportsActionNumber = len(sportsActionTag)

In [18]:
def evaluation( truth, predicted, categoryIndex ):

    # TP,FP,FN,TN indicate True Positive, False Positive, False Negative, True Negative respectively
    TP = 1
    FP = 1
    FN = 1
    TN = 1

    # Categories are Sports Action 1=>0, Sports Action 2=> 1, Sports Action 3=>2  etc..
    for fIndex in range(len(truth)):

         # Positive prediction for each feature
        if ( int(predicted[fIndex]) == categoryIndex):
            # TP=> when P[i] = T[i] = Ci
            if (int(truth[fIndex]) == int (predicted[fIndex])):
                TP += 1
            else:
                FP += 1
        else: # Negative Prediction
            if ( int ( truth[fIndex]) == categoryIndex ):
                FN += 1
            else:
                TN += 1


    # Calculate Sensitivity - True Positive Rate - Recall
    sensitivity = TP / float ( TP + FN )

    # Specificity - True Negative Rate
    specificity = TN / float ( TN + FP )

    #Calculate accuracy
    accuracy =  ( TP + TN ) / float ( TP + FP + FN + TN )


    return sensitivity, specificity, accuracy

In [19]:
def getSportsActionName(saIndex):

    keys   = sportsActionTag.keys()

    for key in keys:
        if saIndex == sportsActionTag[key]:
            return key

In [20]:
print(sportsActionFeatures.shape)

(2234426, 129)


In [21]:
featureAndLabelList=sportsActionFeatures
np.random.shuffle(featureAndLabelList)

In [22]:
subsetLength =  int(len(featureAndLabelList)*0.8)
trainigSet = featureAndLabelList[:subsetLength]
valdationSet = featureAndLabelList[subsetLength : ]

In [23]:
# Get all features in a array
label = []
feature = []
for featureAndLabel in trainigSet:
    label.append(int(featureAndLabel[0]))
    feature.append((np.delete(featureAndLabel, 0)).tolist())

In [24]:
# Train model
print("INFO: Training ... ")
clf=RandomForestClassifier(n_estimators=13)
clf=clf.fit(feature,label)

# Prepare validation feature and label to be predicted
# print("INFO: Prediction for ", getSportsActionName(rIndex))
vFeatureList = []
vLabelList   = [] # Ground Truth
for featureAndLabel in valdationSet:
    vFeatureList.append(featureAndLabel[1:].tolist())
    vLabelList.append(featureAndLabel[0])

INFO: Training ... 


In [25]:
# Predict the class label for Validation Feature List
predictedLabel = clf.predict(vFeatureList)

# predict validation set and calculate accuracy
print("INFO: Evaluating ... ")

# Evaluation < Truth>, <Predicted>, <Sports Action Index>
(sen, spec , accu ) = evaluation(vLabelList , predictedLabel.tolist() , 0)
(sen1, spec1 , accu1 ) = evaluation(vLabelList , predictedLabel.tolist() , 1)

INFO: Evaluating ... 


In [26]:
sensitivity = 0.0
specificity = 0.0
accuracy    = 0.0

sensitivity += sen
specificity += spec
accuracy    += accu

sensitivity += sen1
specificity += spec1
accuracy    += accu1
# print("\t   Sensitivity : ", sen)
# print("\t   Specificity : ", spec)
# print("\t   Accuracy    : ", accu)
# Average evaluation metrics
avgSensitivity = sensitivity / sportsActionNumber
avgSpecificity = specificity / sportsActionNumber
avgAccuracy = accuracy / sportsActionNumber


print("  *** Overall Evaluation ***")
print("    Average Sensitivity: ", avgSensitivity)
print("    Average Specificity: ", avgSpecificity)
print("    Average Accuracy   : ", avgAccuracy)

  *** Overall Evaluation ***
    Average Sensitivity:  0.5754917958650128
    Average Specificity:  0.5754917958650128
    Average Accuracy   :  0.697708608382376


In [ ]:
from sklearn import svm
print("INFO: Training ... ")
clf = svm.SVC(gamma=0.01, C=13)
clf.fit(feature,label)

INFO: Training ... 


In [ ]:
# Predict the class label for Validation Feature List
predictedLabel = clf.predict(vFeatureList)

# predict validation set and calculate accuracy
print("INFO: Evaluating ... ")

# Evaluation < Truth>, <Predicted>, <Sports Action Index>
(sen, spec , accu ) = evaluation(vLabelList , predictedLabel.tolist() , 0)
(sen1, spec1 , accu1 ) = evaluation(vLabelList , predictedLabel.tolist() , 1)

In [ ]:
# Evaluation Metrics
sensitivity = 0.0
specificity = 0.0
accuracy    = 0.0

sensitivity += sen
specificity += spec
accuracy    += accu

sensitivity += sen1
specificity += spec1
accuracy    += accu1
# print("\t   Sensitivity : ", sen)
# print("\t   Specificity : ", spec)
# print("\t   Accuracy    : ", accu)
# Average evaluation metrics
avgSensitivity = sensitivity / sportsActionNumber
avgSpecificity = specificity / sportsActionNumber
avgAccuracy = accuracy / sportsActionNumber


print("  *** Overall Evaluation ***")
print("    Average Sensitivity: ", avgSensitivity)
print("    Average Specificity: ", avgSpecificity)
print("    Average Accuracy   : ", avgAccuracy)